# Technical Project Management Round 2

Data:
- [Slack Pings](https://app.dataannotation.tech/publish/g/Y30OIG9YCwtDZSQjHSBWCiY0FTcPWj8AJTcyXyALBnsxDHV8MT1eDkM1JkA=)
- [Emails](https://app.dataannotation.tech/publish/g/Y30OIG9YCwtDZSQjHSBWCiY0FTcPWj8AJTcyXyALBnsxDHV8MT1eDkM1JkA=)
- [Meeting Notes](https://app.dataannotation.tech/publish/g/Y3lDAwBfBjdUaT0vbGJNHwE_RwsTVDdbNW5MYA1AOlsAM0BSExlRDiAbHi0=)
- [Calendar Invites](https://app.dataannotation.tech/publish/g/Y1ElOXJlOwcAdioydCAJNXgQRQZUL2N9eSs_DEtzMG8yNERhIyNRIRw_JjE=)
- [Spec Docs](https://app.dataannotation.tech/publish/g/Y0EOBU1BMiB0R0BaP2R_NiZESHAKVAJ6HhMwUzZRHUA4KlpjAwANPRQacTk=)
- [Tickets](https://app.dataannotation.tech/publish/g/Y38EGgFafB5-C18NHTYJInoLPXdVFyNFDncgcg51FW9VLgwLdTIiLEMHJEQ=)

In [21]:
import AppendPath
from pathlib import Path
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown
import keyring

In [22]:
def authenticate() -> tuple[requests.sessions.Session, requests.Response]:
    session = requests.Session()

    # Step 1: Load login page to get CSRF token
    login_url = "https://app.dataannotation.tech/users/sign_in"
    login_page = session.get(login_url)
    soup = BeautifulSoup(login_page.text, "html.parser")

    # Step 2: Extract CSRF token
    token_input = soup.find("input", {"name": "authenticity_token"})
    auth_token = token_input["value"] if token_input else None

    # Step 3: Build login payload
    payload = {
        "user[email]": keyring.get_password("DataAnnotation", "email"),
        "user[password]": keyring.get_password("DataAnnotation", "password"),
        "authenticity_token": auth_token,
        "commit": "Login",
    }

    # Step 4: Submit login form
    response = session.post(login_url, data=payload)
    return (session, response)

session, response = authenticate()

## Load Data

If files exist, load directly from file, otherwise load from network and save

In [23]:
dirname = Path("./Interview/TPM 2")
document_types = ["Instructions", "Slack", "Emails", "Meetings", "Calendar", "Spec", "Tickets"]
base_url = "https://app.dataannotation.tech/publish/g/"
urls = {
    "Instructions":"Y18uOWhlBmFjeygTDmB9FSgfHSY8CAIDFiYTcg10ZGdSBwVjCENbewQDLDU=",
    "Slack": "Y30OIG9YCwtDZSQjHSBWCiY0FTcPWj8AJTcyXyALBnsxDHV8MT1eDkM1JkA=",
    "Emails": "Y38iGkFNFxZySgovFmAIFzhAKxQTJjhdFXcMCQANJWsjIm5fLANYK0AwBxc=",
    "Meetings": "Y3lDAwBfBjdUaT0vbGJNHwE_RwsTVDdbNW5MYA1AOlsAM0BSExlRDiAbHi0=",
    "Calendar": "Y1ElOXJlOwcAdioydCAJNXgQRQZUL2N9eSs_DEtzMG8yNERhIyNRIRw_JjE=",
    "Spec": "Y0EOBU1BMiB0R0BaP2R_NiZESHAKVAJ6HhMwUzZRHUA4KlpjAwANPRQacTk=",
    "Tickets": "Y38EGgFafB5-C18NHTYJInoLPXdVFyNFDncgcg51FW9VLgwLdTIiLEMHJEQ=",
}

documents = {}
for doc_type in document_types:
    filename = dirname.joinpath(doc_type + ".txt")
    if filename.exists():
        text = filename.read_text(encoding="UTF8")
    else:
        url = base_url + urls[doc_type]
        response = session.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.body.get_text(separator="\n", strip=True)
        filename.write_text(text, encoding="UTF8")
    documents[doc_type] = text